In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from itertools import product
from ipywidgets import IntProgress
from IPython.display import display
import warnings

warnings.filterwarnings('ignore')

In [38]:
# download data
train = pd.read_csv('train_kaggle.csv')
test = pd.read_csv('test_kaggle.csv')
train['Date'] = pd.to_datetime(train['Date'], dayfirst = True)
test['Date'] = pd.to_datetime(test['Date'], dayfirst = True)
train.set_index(['Date'], inplace=True)
test.set_index(['Date'], inplace=True)

In [39]:
train['Promo'] = train['Promo'].fillna(0)
train.reset_index().set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)
train['Regular_Price'] = train['Regular_Price'].ffill().bfill()
train.reset_index().set_index(['Date'], inplace=True)

# add actual price (promo price when promo occurred or regular price otherwise)

train['Actual_Price'] = train.Promo_Price.combine_first(train.Regular_Price) 
train.head()

,Store_id,SKU_id,Promo,Demand,Regular_Price,Promo_Price,Actual_Price
Date,,,,,,,
2015-01-01,1,1,0.0,22,163.78,NaN,163.78
2015-01-02,1,1,0.0,41,163.78,NaN,163.78
2015-01-03,1,1,0.0,35,163.78,NaN,163.78
2015-01-04,1,1,0.0,72,163.78,NaN,163.78
2015-01-05,1,1,0.0,25,163.78,NaN,163.78


In [40]:
test['Promo'] = test['Promo'].fillna(0)
test.reset_index().set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)
test['Regular_Price'] = test['Regular_Price'].ffill().bfill()
test.reset_index().set_index(['Date'], inplace=True)

# add actual price (promo price when promo occurred or regular price otherwise)

test['Actual_Price'] = test.Promo_Price.combine_first(test.Regular_Price) 
test.head()

,Store_id,SKU_id,Promo,Demand,Regular_Price,Promo_Price,Actual_Price
Date,,,,,,,
2016-05-23,1,1,1.0,NaN,128.98,119.6,119.60
2016-05-24,1,1,0.0,NaN,128.98,NaN,128.98
2016-05-25,1,1,0.0,NaN,131.70,NaN,131.70
2016-05-26,1,1,0.0,NaN,131.70,NaN,131.70
2016-05-27,1,1,0.0,NaN,131.70,NaN,131.70


In [41]:
train.reset_index(inplace=True)
train["weekday"] = train.Date.dt.weekday
train["monthday"] = train.Date.dt.day
train['is_weekend'] = train.weekday.isin([5, 6]) * 1

test.reset_index(inplace=True)
test["weekday"] = test.Date.dt.weekday
test["monthday"] = test.Date.dt.day
test['is_weekend'] = test.weekday.isin([5, 6]) * 1

In [42]:
data = pd.concat([train, test]).set_index('Date').reset_index()
data.head()

,Date,Store_id,SKU_id,Promo,Demand,Regular_Price,Promo_Price,Actual_Price,weekday,monthday,is_weekend
0,2015-01-01,1,1,0.0,22.0,163.78,NaN,163.78,3,1,0
1,2015-01-02,1,1,0.0,41.0,163.78,NaN,163.78,4,2,0
2,2015-01-03,1,1,0.0,35.0,163.78,NaN,163.78,5,3,1
3,2015-01-04,1,1,0.0,72.0,163.78,NaN,163.78,6,4,1
4,2015-01-05,1,1,0.0,25.0,163.78,NaN,163.78,0,5,0


In [43]:
def percentile(n):
    '''Calculate n - percentile of data'''
    def percentile_(x):
        return np.nanpercentile(x, n)
    percentile_.__name__ = 'pctl%s' % n
    return percentile_


def lagged_features(data
                    , target_var = 'demand'
                    , id_columns = ['Store_id']
                    , lags = [7, 14, 21, 28]
                    , windows = [7, 14]
                    , preaggreagation_methods = ['mean'] # ['mean', 'count']
                    , aggregation_methods = ['mean', 'median', percentile(10),  percentile(90)]
                    , filters = None
                    ):
    '''Calculate lagged features '''
    ''' df - data frame
        target_var - column name which is used to calculate lagged features
        by_store - whether calculate stats by stores or not
        id_columns - columns to groups
        lags - 
        windows - list of windows, calculation is performed within time range length of window
        preaggreagation_methods - applied methods before rolling
        aggregation_methods - method of aggregation, e.g. 'mean', 'median', 'std', etc.
        filter = dict of dict: {<column_name>:{'postfix':'condition of the filter'}}
    '''
    out_df = deepcopy(data)
    df = deepcopy(data)
    true_demand = df['Demand']

    if filters is None:
        filters = {'':{''}}
        
    total = 1
    for k, v in flts.items():
         total *= len(flts[k])
    
    progress = IntProgress(min=0, max=total) # instantiate the bar
    display(progress) # display the bar
    
    keys, values = zip(*filters.items())
    for bundle in product(*values):

        condition = ' & '.join([keys[i] + filters[keys[i]][bundle[i]] for i in range(len(keys))])
        name =  '_'.join([bundle[i] for i in range(len(keys))])
        if len(condition) > 0:
            idx = df.eval(condition)
        else:
            idx = df.index >= 0
        
        if len(df[idx].index) > 0:
            
            df.loc[~idx, 'Demand'] = None
            for w in windows:
                for preaggr in preaggreagation_methods:
                    for method in aggregation_methods:
                        
                        lf_df = df.set_index('Date').sort_index().groupby(id_columns + ['Date']).\
                        agg(preaggr).rolling(window=w, min_periods=1).agg(method)
                            
                        for l in lags:
                            method_name = method.__name__ if type(method) != str else method
                            new_names = {x: "lag{0}_wdw{1}_key{2}_preag{3}_ag{4}_{5}_{6}".
                                         format(l, w, '_'.join(id_columns), preaggr, method_name, x, name) for x in lf_df.columns}

                            out_df = pd.merge(out_df, lf_df['Demand'].shift(l).reset_index().rename(columns = new_names),
                                              how='left', on=id_columns + ['Date'])
        df['Demand'] = true_demand
        progress.value += 1
    return out_df



In [44]:
flts = {'Promo': {'oprm':'>0', 'npromo':'==0', 'aprm':'>-1'}, 'weekday' : {'md':'==0', 'tue':'==1', 'wd':'==2', 'th':'==3', 'fr':'==4', 'sa':'==5', 'su':'==6', 'anyday':'>-1'}}
# data with added lagged features
data_lagged_features = lagged_features(data, target_var = 'Demand'
                    , id_columns = ['SKU_id']
                    , lags = [22]
                    , windows = [14]
                    , preaggreagation_methods = ['mean'] # ['mean', 'count']
                    , aggregation_methods = ['mean', 'median']
                    , filters = flts
                    )

IntProgress(value=0, max=24)

In [45]:
data_lagged_features.isna().sum().sort_values()

Date                                                                 0
Store_id                                                             0
SKU_id                                                               0
Promo                                                                0
Regular_Price                                                        0
is_weekend                                                           0
Actual_Price                                                         0
weekday                                                              0
monthday                                                             0
lag22_wdw14_keySKU_id_preagmean_agmean_Demand_aprm_anyday         2686
lag22_wdw14_keySKU_id_preagmean_agmedian_Demand_aprm_anyday       2686
lag22_wdw14_keySKU_id_preagmean_agmedian_Demand_aprm_th           2902
lag22_wdw14_keySKU_id_preagmean_agmean_Demand_aprm_th             2902
lag22_wdw14_keySKU_id_preagmean_agmedian_Demand_aprm_fr           2975
lag22_

In [46]:
idx = data.eval('Promo>0 & weekday==0')
data[idx]

,Date,Store_id,SKU_id,Promo,Demand,Regular_Price,Promo_Price,Actual_Price,weekday,monthday,is_weekend
53,2015-02-23,1,1,1.0,553.0,149.37,135.64,135.64,0,23,0
81,2015-03-23,1,1,1.0,478.0,140.67,127.49,127.49,0,23,0
102,2015-04-13,1,1,1.0,537.0,133.33,119.33,119.33,0,13,0
144,2015-05-25,1,1,1.0,383.0,131.70,119.33,119.33,0,25,0
165,2015-06-15,1,1,1.0,349.0,131.70,119.33,119.33,0,15,0
...,...,...,...,...,...,...,...,...,...,...,...
92818,2016-06-13,105,2,1.0,NaN,138.50,114.17,114.17,0,13,0
92846,2016-06-13,106,2,1.0,NaN,138.50,114.17,114.17,0,13,0
92856,2016-06-13,107,2,1.0,NaN,138.50,114.17,114.17,0,13,0
92869,2016-06-13,109,2,1.0,NaN,138.50,114.17,114.17,0,13,0
